# Imports

In [1]:
import pandas as pd
import copy
df_raw = pd.read_csv("~/code/moth_project/0_database/tracking/track_localize_classify_annotation-2022_05_13.csv")

In [3]:
def count_species(csv):
    df = pd.read_csv(csv)
    df_count = df.groupby(['subclass']).size().sort_values(ascending=False)
    df_count.to_csv('count.txt',header=False)
    return df_count

def calculate_best_confidence_max_value(df,df_specific_id):
    max_confidence_index = df_specific_id['confidence'].idxmax()
    new_label = df.iloc[max_confidence_index,df.columns.get_loc("subclass")]
    return max_confidence_index,new_label

def modify_label_confidence(df,df_specific_id):
    #print(df_specific_id)
    max_confidence_index,new_label = calculate_best_confidence_max_value(df,df_specific_id)
    print('Selected label ', new_label)
    #df_specific_id = df_specific_id.drop(max_confidence_index)
    for index, row in df_specific_id.iterrows():
        #print(index, max_confidence_index)
        if index != max_confidence_index:
            #print('differences')
            #print(df.iloc[max_confidence_index, df.columns.get_loc("subclass")], df.iloc[max_confidence_index, df.columns.get_loc("confidence")])
            #print(df.iloc[index, df.columns.get_loc("subclass")], df.iloc[index, df.columns.get_loc("confidence")])
            df.iloc[index, df.columns.get_loc("subclass")] = new_label
            df.iloc[index, df.columns.get_loc("confidence")] = 'smoothed'
        #print('post modif', df.iloc[index, df.columns.get_loc("subclass")], df.iloc[index, df.columns.get_loc("confidence")])
        

def smoothing_confidence_with_tracking(df_raw):
    df = df_raw.copy()
    max_id_tracked = int(df['track_id'].max())
    #max_id_tracked = 3
    for track_id in range(1,max_id_tracked+1):
        df_ids_raw = df[df['track_id'] == track_id]
        if df_ids_raw.shape[0] > 1:
            print("\n", df_ids_raw[['subclass','confidence']])
            df_specific_id = df_ids_raw.astype({'confidence': int}) # necessary, otherwise: confidence = object instead of int
            modify_label_confidence(df,df_specific_id)
    df_final = df.mask(df == '')
    return df_final

In [4]:
df_modified = smoothing_confidence_with_tracking(df_raw)
# df_modified.to_csv('test_counting.csv')
# count_species('test_counting.csv')


                     subclass  confidence
109      Hyalophora cecropia          89
114     Acleris flavivittana          20
119  Korscheltellus gracilis          62
Selected label  Hyalophora cecropia

                  subclass confidence
153  Callopistria cordata         23
158  Callopistria cordata         34
Selected label  Callopistria cordata

                 subclass confidence
175        Catocala ilia         58
180  Pyrausta orphisalis         16
Selected label  Catocala ilia

                     subclass confidence
195       Hypoprepia miniata         22
200  Agriphila vulgivagellus         35
205      Archips argyrospila         15
Selected label  Agriphila vulgivagellus

                    subclass confidence
663      Papaipema pterisii          2
669          Anopina ednana          8
677     Drasteria adumbrata          5
684           Catocala ilia         13
691           Catocala ilia         11
697           Catocala ilia          6
704  Chytonix palliatricula    

In [5]:
# print(df_raw.iloc[114])
# print(df_modified.iloc[114])

In [6]:
df_modified.to_csv('test.csv')